# Feature Selection in Machine-Learning

## Refs:
http://scikit-learn.org/stable/modules/feature_selection.html


In [6]:
import os, sys
import pickle
import numpy as np

from numpy import genfromtxt

import pandas as pd
#import pandas.io.data
from pandas_datareader import data, wb
from pandas import DataFrame



%matplotlib inline
import matplotlib.pyplot as plt
from pylab import rcParams
rcParams['figure.figsize'] = 10, 10 # increase plot size a bit...
rcParams['axes.formatter.useoffset'] = False  # disable scalar formatter / offset in axes labels

In [7]:
# import your own modules by providing path
sys.path.append('../python')

import apply_detree

#import imp; imp.reload(apply_detree)  # force the interpreter to re-import if apply_detree.py changed

In [8]:
my_file_csv='/g/data/u46/fxz547/wofs_training_sample_data/all_samples.csv'
pdf = apply_detree.get_pdf_from_csv(my_file_csv)
# todo: keep the old class_label. how to add a new_class column?
pdf.shape

(4937172, 17)

In [11]:

#  Assign W_* into Water; all other classes into NotWater. Binary classes
pdf['myclass'] = 0 # add a new column myclass
# change the column myclass to datatype uint8
pdf[['myclass']] =pdf[['myclass']].astype('uint8')

w_condition= pdf.Class_name.str.startswith('W') | pdf.Class_name.str.endswith('_Water')   #match('^W_*') and CloudShadow_Water

pdf.loc[w_condition, 'myclass'] = 128 #'Water'


In [12]:
pdf['myclass'].dtypes

dtype('uint8')

In [13]:
# Group by Pixel Class_name, then count the number of rows in each group
pdf.groupby('myclass').count()

,Bright,Class_name,Mean_PQMas,Mean_tm1c,Mean_tm2c,Mean_tm3c,Mean_tm4c,Mean_tm5c,Mean_tm7c,NDI_43,NDI_52,NDI_54,NDI_72,NDI_74,Scene_Name,X_Min_Pxl,Y_Min_Pxl
myclass,,,,,,,,,,,,,,,,,
0,3988944,3988944,3988944,3988944,3988944,3988944,3988944,3988944,3988944,3988944,3988944,3988944,3988944,3988944,3988944,3988944,3988944
128,948228,948228,948228,948228,948228,948228,948228,948228,948228,948228,948228,948228,948228,948228,948228,948228,948228


In [14]:
dfwater=pdf[pdf['myclass'] ==128]
dfnowater= pdf[pdf['myclass'] == 0]

In [15]:
print (dfwater.shape)
print(dfnowater.shape)

(948228, 18)
(3988944, 18)


In [17]:
clm_list = []
for column in pdf.columns: clm_list.append(column)
    
print clm_list

['Bright', 'Class_name', 'Mean_PQMas', 'Mean_tm1c', 'Mean_tm2c', 'Mean_tm3c', 'Mean_tm4c', 'Mean_tm5c', 'Mean_tm7c', 'NDI_43', 'NDI_52', 'NDI_54', 'NDI_72', 'NDI_74', 'Scene_Name', 'X_Min_Pxl', 'Y_Min_Pxl', 'myclass']


In [20]:
# select 3 most important feature attributes as predictive columns
# Note that the order of these variables does not matter. The same decision tree will be arrived
pred_cols =clm_list[3:14]

print pred_cols

['Mean_tm1c', 'Mean_tm2c', 'Mean_tm3c', 'Mean_tm4c', 'Mean_tm5c', 'Mean_tm7c', 'NDI_43', 'NDI_52', 'NDI_54', 'NDI_72', 'NDI_74']


In [21]:
#select the columns values into numpy array

X= pdf[pred_cols].values

y = pdf['myclass'].values  # the new column myclass
       
print (X.shape, type(X), X.dtype)
print (y.shape, y.dtype)

((4937172, 11), <type 'numpy.ndarray'>, dtype('float64'))
((4937172,), dtype('uint8'))


In [22]:
from sklearn.datasets import load_iris
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
# iris = load_iris()
# X, y = iris.data, iris.target
X.shape
X_new = SelectKBest(chi2, k=2).fit_transform(X, y)
X_new.shape

ValueError: Input X must be non-negative.

In [4]:
X[:10]

array([[ 5.1,  3.5,  1.4,  0.2],
       [ 4.9,  3. ,  1.4,  0.2],
       [ 4.7,  3.2,  1.3,  0.2],
       [ 4.6,  3.1,  1.5,  0.2],
       [ 5. ,  3.6,  1.4,  0.2],
       [ 5.4,  3.9,  1.7,  0.4],
       [ 4.6,  3.4,  1.4,  0.3],
       [ 5. ,  3.4,  1.5,  0.2],
       [ 4.4,  2.9,  1.4,  0.2],
       [ 4.9,  3.1,  1.5,  0.1]])

In [5]:
y[:10]

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

In [26]:

# Recursive Feature Elimination
from sklearn import datasets
from sklearn.feature_selection import RFE
from sklearn.linear_model import LogisticRegression
# load the iris datasets
dataset = datasets.load_iris()
# create a base classifier used to evaluate a subset of attributes
model = LogisticRegression()
# create the RFE model and select 3 attributes
rfe = RFE(model, 3)
#Orig  rfe = rfe.fit(dataset.data, dataset.target)
rfe = rfe.fit(X, y)
# summarize the selection of the attributes
print(rfe.support_)
print(rfe.ranking_)

[False False False False False False  True  True  True False False]
[4 5 7 8 9 6 1 1 1 2 3]


In [27]:
from sklearn import datasets
from sklearn import metrics
from sklearn.ensemble import ExtraTreesClassifier
# load the iris datasets
dataset = datasets.load_iris()
# fit an Extra Trees model to the data
model = ExtraTreesClassifier()

### model.fit(dataset.data, dataset.target)
model.fit(X, y)

# display the relative importance of each attribute
print(model.feature_importances_)

[ 0.02654746  0.02281372  0.02931091  0.06198698  0.16721007  0.09394792
  0.08071115  0.22584792  0.01187079  0.16850388  0.1112492 ]


In [44]:
from sklearn import tree
clf = tree.DecisionTreeClassifier(max_depth=7)

clf = clf.fit(X, y)
print(clf.feature_importances_)

[  7.74101362e-03   1.05949249e-03   1.75101183e-04   6.91431169e-02
   1.95543213e-02   2.44999753e-03   1.12615914e-02   8.69358787e-01
   1.94898528e-03   1.63440017e-02   9.63591641e-04]


In [ ]:
from sklearn import tree
clf = tree.DecisionTreeClassifier(max_depth=5)

In [42]:
# Traing the model with the sample data %time clf = clf.fit(X, Y)

clf = clf.fit(X, y)

In [43]:
# max_dep=5
print(clf.feature_importances_)

[  4.88014240e-03   5.70562094e-04   0.00000000e+00   6.58228035e-02
   1.97781439e-02   0.00000000e+00   8.76354758e-03   8.86324686e-01
   0.00000000e+00   1.33066141e-02   5.53500195e-04]


In [39]:
# max_dep=3
print(clf.feature_importances_)

[ 0.          0.          0.          0.06266062  0.02035376  0.          0.
  0.91206517  0.          0.00492045  0.        ]


In [40]:
clf.classes_

array([  0, 128], dtype=uint8)

In [ ]:
It looks like R52 B4 B5  R72 are the most important relevant variables

In [35]:
np.mean(X[:, 5])

1321.1676261228088